
# 1. Introdução e Metadados

### 1.1. Contexto e Fontes de Dados

Os dados foram obtidos a partir da plataforma [Kaggle](https://www.kaggle.com/), em uma competição de predição de malware da Microsoft.

- **Fonte Principal:** [Microsoft Malware Prediction](https://www.kaggle.com/competitions/microsoft-malware-prediction/data).

- **Fonte Original:** Os dados são derivados de telemetria de máquinas Windows, coletados pela Microsoft para identificar padrões de malware.

### Sobre a Competição

A competição Microsoft Malware Prediction tem como objetivo prever se uma máquina será infectada por malware com base em características do sistema operacional, hardware e software instalado. Esta é uma tarefa de classificação binária onde o target é a variável `HasDetections`.

### 1.2. Sobre o Dataset e Estrutura

O dataset descreve características de máquinas Windows e sua propensão a serem infectadas por malware. As informações abrangem várias áreas principais:

- **Características do Sistema:** Informações sobre o sistema operacional, versão, arquitetura e configurações.
- **Hardware:** Dados sobre processador, memória RAM, espaço em disco e outros componentes.
- **Software:** Informações sobre programas instalados, versões e configurações.
- **Telemetria:** Dados de uso e comportamento do sistema.

A escolha deste conjunto de dados se deve à sua relevância para segurança cibernética e ao desafio de classificação binária que apresenta. Este dataset combina um grande volume de registros com alta dimensionalidade, oferecendo um cenário realista para projetos de Machine Learning em segurança.

### 1.3. Desafios de Qualidade e Limitações Iniciais

Entre as oportunidades de tratamento já identificadas, destacam-se:
- **Alta Dimensionalidade:** O dataset possui um grande número de features, muitas das quais podem ser irrelevantes ou redundantes.
- **Dados Ausentes:** Presença significativa de valores faltantes que precisarão de estratégias de tratamento adequadas.
- **Desbalanceamento de Classes:** Possível desbalanceamento entre classes de malware e não-malware.
- **Engenharia de Features:** Necessidade de criar novas features ou transformar as existentes para melhorar a performance do modelo.


# 2. Análise Exploratória

A seguir, utilizaremos Python para auxiliar na análise dos dados brutos. Com a biblioteca [`Pandas`](https://pandas.pydata.org/), podemos visualizar e alterar os dados utilizando `Dataframes` e `Series`. Uma `Series` é um array unidimensional, e um `Dataframe` é como uma tabela, em que cada coluna é uma `Series`.  

Além dessas bibliotecas, também podemos usar a `seaborn` para desenhar gráficos com base nos dados das `Series` e `Dataframes`.


## 2.1 Importação das bibliotecas e carregamento do Dataset


In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
from matplotlib.ticker import MaxNLocator
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# Configurações para melhor visualização
plt.style.use('default')
sns.set_palette("husl")
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)


In [ ]:
# Carregamento do dataset com otimizações para arquivos grandes
df = pd.read_csv("./train.csv", sep=",", encoding='utf8', low_memory=False)


## 2.2 Head do Dataset


A tabela apresentada mostra as cinco primeiras entradas do nosso conjunto de dados, o que nos permite uma verificação inicial das colunas disponíveis e dos tipos de dados que elas contêm.


In [ ]:
df.head(5)


## 2.3 Dimensões do Dataset


Após a primeira visualização, o próximo passo é verificar as dimensões do nosso conjunto de dados. A "dimensão" de um dataset refere-se à sua estrutura em termos de quantidade de linhas e colunas.

Conforme o resultado abaixo, nosso dataset é composto por um grande número de linhas e colunas, o que nos dá uma noção do volume de dados que iremos analisar.


In [ ]:
linhas, colunas = df.shape

print(f"Número de tuplas: {linhas:,}")
print(f"Número de colunas: {colunas}")


## 2.4 Identificação das Colunas e Tipos de Dados


Agora que conhecemos as dimensões gerais do nosso dataset, o próximo passo é nos aprofundarmos nessas colunas. Precisamos identificar o nome de cada uma das variáveis que temos à disposição para entender com quais informações estamos trabalhando.

O comando a seguir listará todas as colunas, dando uma visão dos atributos disponíveis para cada máquina.


In [ ]:
print(f"Total de colunas: {len(df.columns)}")
print("\nPrimeiras 20 colunas:")
for i, col in enumerate(df.columns[:20]):
    print(f"{i+1:2d}. {col}")

if len(df.columns) > 20:
    print(f"\n... e mais {len(df.columns) - 20} colunas")


Com os nomes das colunas já identificados, vamos entender como os dados de cada variável estão sendo interpretados. Verificar os tipos de dados (dtypes) é um passo importante para garantir a integridade da análise e planejar a etapa de limpeza.

O resultado a seguir nos revela a distribuição dos tipos de dados no dataset.


In [ ]:
# Análise dos tipos de dados
dtype_counts = df.dtypes.value_counts()
print("Distribuição dos tipos de dados:")
print(dtype_counts)

print("\nDetalhamento por tipo:")
for dtype in df.dtypes.unique():
    cols = df.select_dtypes(include=[dtype]).columns.tolist()
    print(f"\n{dtype}: {len(cols)} colunas")
    if len(cols) <= 10:
        print(f"  {cols}")
    else:
        print(f"  {cols[:10]} ... e mais {len(cols)-10} colunas")


## 2.5 Qualidade dos Dados


### 2.5.1 Análise de Valores Ausentes (Nulos/NaN)


#### Dados não nulos

Enquanto o passo anterior nos mostrou apenas os tipos de dados, a análise a seguir também revelará a quantidade de valores não nulos em cada coluna, nos dando um panorama sobre a presença de dados ausentes.

Este diagnóstico serve para identificar lacunas que precisam ser tratadas antes de qualquer análise mais profunda.


In [ ]:
# Informações gerais sobre o dataset
print("Informações gerais do dataset:")
print(f"Memória utilizada: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
print(f"\nResumo de informações:")
df.info()


#### Dados nulos

No passo anterior, o método `info()` nos deu um bom indicativo da presença de dados ausentes. Para abordar esse problema, vamos calcular a soma exata de valores nulos para cada coluna do dataset.


In [ ]:
# Análise de valores nulos
null_counts = df.isnull().sum()
null_percentages = (null_counts / len(df)) * 100

# Criar DataFrame com informações de nulos
null_info = pd.DataFrame({
    'Coluna': null_counts.index,
    'Valores_Nulos': null_counts.values,
    'Percentual_Nulos': null_percentages.values
})

# Ordenar por percentual de nulos
null_info = null_info.sort_values('Percentual_Nulos', ascending=False)

print("Top 20 colunas com mais valores nulos:")
print(null_info.head(20).to_string(index=False))


### 2.5.2 Análise da Variável Target


Para um problema de classificação binária, é fundamental entender a distribuição da variável target. Vamos analisar a variável `HasDetections` que indica se a máquina foi infectada por malware.


In [ ]:
# Análise da variável target
if 'HasDetections' in df.columns:
    target_counts = df['HasDetections'].value_counts()
    target_percentages = df['HasDetections'].value_counts(normalize=True) * 100
    
    print("Distribuição da variável target (HasDetections):")
    print(f"\nContagem:")
    for value, count in target_counts.items():
        print(f"  {value}: {count:,} ({target_percentages[value]:.2f}%)")
    
    # Visualização da distribuição
    plt.figure(figsize=(8, 6))
    ax = sns.countplot(data=df, x='HasDetections', palette=['#5bc0de', '#d9534f'])
    ax.set_title('Distribuição da Variável Target (HasDetections)')
    ax.set_xlabel('HasDetections (0=Sem Malware, 1=Com Malware)')
    ax.set_ylabel('Contagem')
    
    # Adicionar percentuais nas barras
    for i, p in enumerate(ax.patches):
        height = p.get_height()
        ax.text(p.get_x() + p.get_width()/2., height + height*0.01,
                f'{height:,}\n({target_percentages[i]:.1f}%)',
                ha='center', va='bottom')
    
    plt.tight_layout()
    plt.show()
    
    # Verificar balanceamento
    balance_ratio = min(target_counts) / max(target_counts)
    print(f"\nRazão de balanceamento: {balance_ratio:.3f}")
    if balance_ratio < 0.1:
        print("⚠️  Dataset altamente desbalanceado!")
    elif balance_ratio < 0.3:
        print("⚠️  Dataset moderadamente desbalanceado")
    else:
        print("✅ Dataset relativamente balanceado")
        
else:
    print("Variável 'HasDetections' não encontrada no dataset.")
    print("Colunas disponíveis que podem ser targets:")
    potential_targets = [col for col in df.columns if 'detect' in col.lower() or 'target' in col.lower() or 'label' in col.lower()]
    if potential_targets:
        print(potential_targets)
    else:
        print("Nenhuma coluna target óbvia encontrada.")


### Conclusão da Análise Exploratória e Próximos Passos


A fase de Análise Exploratória de Dados foi concluída, nos fornecendo um panorama sobre a estrutura, a qualidade e as características do nosso conjunto de dados. Através desta investigação, embora rico em informações, o dataset possui diversas características que precisam ser consideradas antes de qualquer análise aprofundada.

Os principais achados identificados foram:

- **Dimensões do Dataset:** Um dataset de grande escala com muitas observações e features.
- **Dados Ausentes:** Presença de valores faltantes que precisarão de estratégias de tratamento.
- **Dados Duplicados:** Verificação da existência de registros duplicados.
- **Variável Target:** Análise da distribuição da variável `HasDetections` e verificação de balanceamento.
- **Correlações:** Identificação de variáveis altamente correlacionadas que podem ser redundantes.
- **Variabilidade:** Análise de valores únicos para identificar variáveis constantes ou com baixa variabilidade.

A conclusão desta etapa é que o dataset em seu estado bruto apresenta características típicas de dados de telemetria de sistemas, com alta dimensionalidade e complexidade. Portanto, o próximo passo será focado na **Engenharia de Features** e **Limpeza de Dados**, onde iremos executar um plano para otimizar o dataset para modelagem de Machine Learning.
